### Using manual feature-map

In [251]:
from collections import defaultdict
from collections import Counter
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error

def remapOnFeatureMap2(data, featureMap2):
    remapOnFeatureMap2 = []
    
    for i, row in enumerate(data):
        newRow = []
        for j, d in enumerate(row):
            if j not in [0,7]:
                if (j,d) in featureMap2:
                    newRow.append(featureMap2[(j,d)])
        remapOnFeatureMap2.append(newRow)
    
    binRemapped2 = np.zeros((len(data), len(featureMap2)), dtype=float)
    for i, row in enumerate(remapOnFeatureMap2):
        for j, d in enumerate(row):
            if j not in [0,7]:
                binRemapped2[i][d] = 1
            else:
                binRemapped2[i][j] = d
        binRemapped2[i][-1] = 1
    return binRemapped2

def removeNA(data):
    for i, row in enumerate(data):
        for j, d in enumerate(row):
            if d=='NA':
                data[i][j] = 0
    return data

def removeNAFromDF(data):
    for col in data.columns:
        if col not in binTrainDF.columns:
            data = data.drop(col, axis = 1)
    print('After dropping extra cols from data ----- size of binTrainDF = {} and dataDF = {}'.format(binTrainDF.shape, data.shape))
    
    for col in binTrainDF.columns:
        if col not in data.columns:
            data[col] = 0
    print('After setting cols in data which are not in train to zero ----- size of binTrainDF = {} and dataDF = {}'.format(binTrainDF.shape, data.shape))
    
    for col in data.columns:
        data[col] = data[col].fillna(data[col].mean())
#         data[col] = data[col].fillna(0)
    
    return data

def writePredictions(ids, preds):
    final = [str(i)+','+str(p) for i,p in zip(ids, preds)]
    with open('submission.csv', 'w') as output:
        output.write('Id,SalePrice\n')
        for x in final:
            output.write(str(x))
            output.write('\n')

if __name__ == '__main__':
    trainFile, devFile, testFile = 'train.csv', 'my_dev.csv', 'test.csv'
    
    train = np.array([s.strip().split(',')[1:-1] for s in open(trainFile).readlines()][1:])
    dev = np.array([s.strip().split(',')[1:-1] for s in open(devFile).readlines()][1:])
    test = np.array([s.strip().split(',')[1:] for s in open(testFile).readlines()][1:])
#     print('len of dev {} and len of test {}'.format(len(dev), len(test)))

    header = np.array([s.strip().split(',')[1:-1] for s in open(trainFile).readlines()][0])
#     print([(h,val) for h, val in enumerate(header)])
    
    trainIDs = [s.strip().split(',')[0] for s in open(trainFile).readlines()][1:]
    devIDs = [s.strip().split(',')[0] for s in open(devFile).readlines()][1:]
    testIDs = [s.strip().split(',')[0] for s in open(testFile).readlines()][1:]
    
#     trainY = map(float, [s.strip().split(',')[-1] for s in open(trainFile).readlines()][1:])
    trainY = map(lambda s : np.log(float(s)), [s.strip().split(',')[-1] for s in open(trainFile).readlines()][1:])
    trainY = np.array(list(trainY))
#     devY = map(float, [s.strip().split(',')[-1] for s in open(devFile).readlines()][1:])
    devY = map(lambda s : np.log(float(s)), [s.strip().split(',')[-1] for s in open(devFile).readlines()][1:])
    devY = np.array(list(devY))

    train = removeNA(train)
    dev = removeNA(dev)
    
    featureMap = defaultdict()
    featureRemap = defaultdict()
    featureCategories = defaultdict(list)
    for i, row in enumerate(train):
        for j, d in enumerate(row):
            feature = (j,d)
            if feature not in featureMap:
                featureMap[feature] = len(featureMap)
                featureRemap[len(featureMap)-1] = (feature)
                featureCategories[j].append(d)
            
#     featureMap[(79,0)] = len(featureMap)
#     featureRemap[len(featureMap)-1] = (79,0)
#     featureCategories[79].append(0)
    
#     print('Len of feature map: {}'.format(len(featureMap)))
#     print(featureRemap)
#     print(featureMap)
#     print([len(x[1]) for x in featureCategories.items()])
#     [print('x0: {} x1: {}'.format(x[0], x[1])) for x in featureCategories.items()]
    
#     binTrain = remapOnFeatureMap2(train, featureMap)
#     binDev = remapOnFeatureMap2(dev, featureMap)
#     binTest = remapOnFeatureMap2(test, featureMap)
    
#     model = LinearRegression().fit(X=binTrain, y=trainY)
#     coefs = model.coef_
#     intercept = model.intercept_
#     preds = model.predict(binTest)
    
#     print('len(coefs): {} coefs: {}'.format(len(coefs), coefs))
#     sortedCoefs = np.argsort(coefs)
#     print('Sorted most positive coefs: {}'.format([featureRemap[coef] for coef in sortedCoefs][:10]))
#     print('Sorted most negative coefs: {}'.format([featureRemap[coef] for coef in sortedCoefs][-10:]))
#     print('mse: {}'.format(np.sqrt(mean_squared_error(devY, preds))))
#     print('intercept: {}'.format(intercept))
    
#     preds = [np.exp(s) for s in preds]
#     print('rmse: {}'.format(mean_squared_log_error(devY, preds)))
#     print('predictions after exponentiation: {}'.format(list(zip(preds,np.exp(devY)))))

#     writePredictions(testIDs, preds)

    trainDF, devDF, testDF = pd.read_csv(trainFile), pd.read_csv(devFile), pd.read_csv(testFile)
    trainYDF, devYDF = list(map(lambda s: np.log(s), trainDF['SalePrice'])), list(map(lambda s : np.log(s), devDF['SalePrice']))
    trainDF, devDF, testDF = trainDF.drop(['Id','SalePrice'], axis = 1), devDF.drop(['Id','SalePrice'], axis = 1), testDF.drop(['Id'], axis = 1)
    
#     print('numeric cols: {}'.format(trainDF.select_dtypes(np.number).columns))
#     print('non-numeric cols: {}'.format(trainDF.select_dtypes(exclude=np.number).columns))
    
#     num = [2,3,18,19,25,33,35,36,37,42,43,45,58,61,65,66,67,69]
#     cat = [0,1,4,5,6,7,8,9,10,11,12,13,14,15,16,17,20,21,22,23,24,26,27,28,29,30,31,32,34,38,39,40,41,44,46,47,48,49,50,51,52,53,54,55,56,57,59,60,62,63,64,68,70,71,72,73,74,75,76,77,78]
    
    binTrainDF = pd.concat([trainDF[trainDF.select_dtypes(np.number).columns].fillna(0), pd.get_dummies(trainDF[trainDF.select_dtypes(exclude=np.number).columns]).fillna(0)], axis=1)
    binDevDF = pd.concat([devDF[devDF.select_dtypes(np.number).columns].fillna(0), pd.get_dummies(devDF[devDF.select_dtypes(exclude=np.number).columns]).fillna(0)], axis = 1)
#     binTestDF = pd.concat([pd.get_dummies(testDF[testDF.select_dtypes(np.number).columns]), pd.get_dummies(testDF[testDF.select_dtypes(exclude=np.number).columns])], axis = 1)
#     print('size of binTrainDF = {} and binDevDF = {}'.format(binTrainDF.shape, binDevDF.shape))
#     print(binTestDF.head())
    
    binDevDF = removeNAFromDF(binDevDF)
#     binTestDF = removeNAFromDF(binTestDF)
    
    smartModel = LinearRegression().fit(X=binTrainDF, y=trainYDF)    
#     print('model coefs len: {}'.format(len(smartModel.coef_)))
    smartPreds = smartModel.predict(binDevDF)
    print('rmsle: {}'.format(np.sqrt(mean_squared_error(devYDF, smartPreds))))
    print('bias: {}'.format(smartModel.intercept_))
    print('preds before exponentiation: {}'.format(list(zip(smartPreds, devYDF))))
#     smartPreds = [np.exp(s) for s in smartPreds]
#     devYDF = [np.exp(s) for s in devYDF]
#     print('preds after exponentiation: {}'.format(list(zip(np.exp(smartPreds), np.exp(devYDF)))))
#     writePredictions(testIDs, smartPreds)

After dropping extra cols from data ----- size of binTrainDF = (1460, 288) and dataDF = (146, 224)
After setting cols in data which are not in train to zero ----- size of binTrainDF = (1460, 288) and dataDF = (146, 288)
rmsle: 1.956079917253995
bias: 9.300898790239067
preds before exponentiation: [(9.817638004635576, 11.678439903447801), (11.24985543923975, 12.692503112461443), (10.110978401688715, 11.914047823644255), (9.02750994371888, 11.938193200374572), (10.382139997210443, 12.043553716032399), (7.798660267821924, 11.608235644774552), (10.308463304342164, 12.103486056755072), (11.214058187605724, 11.824079893607152), (10.5397516370016, 12.066810578196666), (10.425946110667816, 11.76718001130949), (7.685526913117499, 11.82700601127654), (10.755801487553441, 12.007560729337492), (8.052940785856338, 11.344506813345266), (11.363323379083777, 12.066810578196666), (8.59724456380836, 12.154779351142624), (10.037049970172188, 12.058152515453552), (9.981195073840935, 11.813030057420567), (

### Using pandas and numpy to create feature-map

In [147]:
from collections import defaultdict
from collections import Counter
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error

def getBinarizedData(data):
    numFeatures = data.select_dtypes(np.number).fillna(0)
    print('Numeric features cols: {}'.format(numFeatures.columns))
#     print(len(numFeatures.columns))
    
    otherFeatures = data.select_dtypes(exclude=np.number).fillna(0)
    print('Other features cols: {}'.format(otherFeatures.columns))
#     print(len(otherFeatures.columns))
    
#     Getting unique values of all non-numeric columns with their corresponding counts
#     print([(train[col].unique(), train[col].nunique()) for col in otherFeatures.columns])

    binData = pd.concat([pd.get_dummies(data[otherFeatures.columns]), data[numFeatures.columns]], axis = 1).fillna(0)
    print('Shape of binarized dataframe: {}'.format(binData.shape))
#     print(binData.head())

    return binData

if __name__ == '__main__':    
    train, dev = pd.read_csv('train.csv'), pd.read_csv('my_dev.csv')
    
    trainY, devY = train.SalePrice.astype('float64'), dev.SalePrice.astype('float64')
    
    train = train.drop(['Id', 'SalePrice'], axis = 1)
    dev = dev.drop(['Id', 'SalePrice'], axis = 1)
    
#     print(train.columns)
#     print(dev.columns)
    
    print('train shape: {}'.format(train.shape))
    print('dev shape: {}'.format(dev.shape))
    
    # To display the dtypes of all columns
#     print(train.dtypes)
    
    binTrain = getBinarizedData(train).astype('float64')
    binDev = getBinarizedData(dev).astype('float64')
#     print(binTrain.dtypes)

    for i, col in enumerate(binTrain.columns):
        isnotavailable = np.any(np.isnan(binTrain[col]))
        if isnotavailable:
            print('isnan is true for col {} and values are {}'.format(col, binTrain[col]))
#         else:
#             print('col - {} nan - {}'.format(col, isnotavailable))
#         print('col - {} inf - {}'.format(col, np.any(np.isfinite(binTrain[col]))))
#         print('col - {} float - {}'.format(col, np.issubdtype(binTrain[col], float)))
#         print('{}: max val in col "{}" is {} '.format(i, col, max(binTrain[col])))
#         print('{}) are values for {} in valid float64 range? {}'.format(i, col, np.any(list(True if val < np.finfo(np.float).max else False for val in binTrain[col]))))

#     for s in trainY:
#         print(trainY[np.isnan(s)==False])
#     print('max value of trainY: {}'.format(max(trainY)))
    
    reg = LinearRegression().fit(binTrain, trainY)
    print('shape of binDev: {}'.format(trainY.shape))
    targetPred = reg.predict(binDev)
    print(mean_squared_log_error(devY, targetPred))

train shape: (1460, 79)
dev shape: (146, 79)
Numeric features cols: Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold'],
      dtype='object')
Other features cols: Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond

ValueError: shapes (146,225) and (288,) not aligned: 225 (dim 1) != 288 (dim 0)